## Objectifs

1. Récupérer la base de lemme + POS du dictionnaire
2. Analyser pour les adjectifs si participes passés
3. Analyser pour les adverbes si
    1. Ablatif d'un nom
    2. -e
4. Analyser pour les noms si dérivés d'un adjectifs
5. Analyser les lemmes au pluriel uniquement (ae, i, es ?)
6. Analyser (jaccard ?) les distances nominatifs / 1ere personne ?

In [1]:
from pandas import DataFrame

## Récupération des lemmes + POS

In [2]:
from collections import defaultdict
DICTIONARY = defaultdict(list)
with open("../../../forcellini-lemmas/dictionary.tsv") as f:
    header = []
    for line_no, line in enumerate(f):
        line = line.strip().split("\t")
        if line_no == 0:
            header = line
            continue
        line = dict(zip(header, line))
        lemma = line["lemma"]
        if lemma[-1].isnumeric():
            lemma = lemma[:-1]
        DICTIONARY[line["POS"]].append(lemma)

## Import Collatinus

In [3]:
from pycollatinus import Lemmatiseur
lemmatizer = Lemmatiseur()

## Analyser pour les adjectifs si participes passés

In [4]:
ADJ_equivalencies = [] # List Tuple
for token in DICTIONARY["ADJqua"]:
    for possibility in lemmatizer.lemmatise(token):
        # {'form': 'abatos', 'lemma': 'Abatos', 
        #    'morph': 'nominatif singulier', 'radical': 'Abat', 'desinence': 'os'}
        if "participe" in possibility["morph"] and "nominatif" in possibility["morph"]:
            ADJ_equivalencies.append((token, possibility["lemma"]))
            break

In [5]:
print(f"{len(ADJ_equivalencies)} adjectifs sont des participes lexicalisés, " 
        f"soit {len(ADJ_equivalencies)/len(DICTIONARY['ADJqua'])*100:.2f} % des adjectifs")
      
ADJ_ARE_VERBS = len(ADJ_equivalencies) / len(DICTIONARY['ADJqua'])

715 adjectifs sont des participes lexicalisés, soit 12.77 % des adjectifs


## Analyser pour les adverbes = ablatif d'un nom

In [6]:
ADV_AS_ABL = [] # List Tuple
for token in DICTIONARY["ADV"]:
    for possibility in lemmatizer.lemmatise(token):
        # {'form': 'abatos', 'lemma': 'Abatos', 
        #    'morph': 'nominatif singulier', 'radical': 'Abat', 'desinence': 'os'}
        if "ablatif" in possibility["morph"] and "singulier" in possibility["morph"]:
            ADV_AS_ABL.append((token, possibility["lemma"]))
            break
            
print(f"{len(ADV_AS_ABL)} adverbes sont des ablatifs lexicalisés, " 
        f"soit {len(ADV_AS_ABL)/len(DICTIONARY['ADV'])*100:.2f} % des adverbes")
      
ADV_ARE_ABL = len(ADV_AS_ABL) / len(DICTIONARY['ADV'])

179 adverbes sont des ablatifs lexicalisés, soit 12.67 % des adverbes


## Analyser pour les adverbes = formation classique ADJ-US+E

In [7]:
ADV_AS_CASE_1e = [] # List Tuple
ADV_AS_CASE_2e = [] # List Tuple
for token in DICTIONARY["ADV"]:
    if token.endswith("e"):  # E = +1ere classe, -e + i = genitif
        for possibility in lemmatizer.lemmatise(token[:-1]+"i"):
            # {'form': 'abatos', 'lemma': 'Abatos', 
            #    'morph': 'nominatif singulier', 'radical': 'Abat', 'desinence': 'os'}
            if "génitif" in possibility["morph"] and "singulier" in possibility["morph"] and \
                "masculin" in possibility["morph"]:
                ADV_AS_CASE_1e.append((token, possibility["lemma"]))
                break
    elif token.endswith("iter"):  # E = +1ere classe, -e + i = genitif
        for possibility in lemmatizer.lemmatise(token[:-4]+"is"):
            # {'form': 'abatos', 'lemma': 'Abatos', 
            #    'morph': 'nominatif singulier', 'radical': 'Abat', 'desinence': 'os'}
            if "génitif" in possibility["morph"] and "singulier" in possibility["morph"] and \
                "masculin" in possibility["morph"]:
                ADV_AS_CASE_2e.append((token, possibility["lemma"]))
                break
            
print(f"{len(ADV_AS_CASE_1e)} adverbes sont des dérivations lexicalisés sur les adjectifs de 1ere classe, " 
      f"{len(ADV_AS_CASE_2e)} adverbes sont des dérivations lexicalisés sur les adjectifs de 2eme classe, " 
        f"soit {(len(ADV_AS_CASE_1e)+len(ADV_AS_CASE_2e))/len(DICTIONARY['ADV'])*100:.2f} % des adverbes")
      
ADV_ARE_CASE = (len(ADV_AS_CASE_1e)+len(ADV_AS_CASE_2e)) / len(DICTIONARY['ADV'])

483 adverbes sont des dérivations lexicalisés sur les adjectifs de 1ere classe, 124 adverbes sont des dérivations lexicalisés sur les adjectifs de 2eme classe, soit 42.96 % des adverbes


## Reste des adverbes

In [8]:
advs = set(DICTIONARY["ADV"])
advs = advs.difference(set([k[0] for k in ADV_AS_ABL]))
advs = advs.difference(set([k[0] for k in ADV_AS_CASE_1e]))
advs = advs.difference(set([k[0] for k in ADV_AS_CASE_2e]))
#print(sorted(list(advs)))

## Analyser pour les noms si dérivés d'un adjectifs

In [9]:
NOM_AS_ADJ = []
for token in DICTIONARY["NOMcom"]+DICTIONARY["NOMpro"]:
    if token in DICTIONARY["ADJ"]:
        NOM_AS_ADJ.append(token)
            
print(f"{len(NOM_AS_ADJ)} noms des adjectifs directement lexicalisés, " 
        f"soit {len(NOM_AS_ADJ)/len(DICTIONARY['NOMcom']+DICTIONARY['NOMpro'])*100:.2f} % des noms communs")
      
NOM_ARE_ADJ = len(NOM_AS_ADJ) / len(DICTIONARY['NOMcom']+DICTIONARY["NOMpro"])

0 noms des adjectifs directement lexicalisés, soit 0.00 % des noms communs


## Analyser les lemmes au pluriel uniquement (ae, i, es ?)

In [10]:
NOM_AS_ADJ = []
for token in DICTIONARY["NOMcom"]+DICTIONARY["NOMpro"]:
    for possibility in lemmatizer.lemmatise(token):
        # {'form': 'abatos', 'lemma': 'Abatos', 
        #    'morph': 'nominatif singulier', 'radical': 'Abat', 'desinence': 'os'}
        if "nominatif" in possibility["morph"] and (
            "masculin" in possibility["morph"] or
            "féminin" in possibility["morph"] or
            "neutre" in possibility["morph"]
        ):
            NOM_AS_ADJ.append((token, possibility["lemma"]))
            break
            
print(f"{len(NOM_AS_ADJ)} noms sont rentrent en collision avec des nominatifs d'adjectifs ou de participes, " 
        f"soit {len(NOM_AS_ADJ)/len(DICTIONARY['NOMcom']+DICTIONARY['NOMpro'])*100:.2f} % des noms communs")
      
NOM_ARE_ADJ = len(NOM_AS_ADJ) / len(DICTIONARY['NOMcom']+DICTIONARY["NOMpro"])

#print(NOM_AS_ADJ)
print(NOM_AS_ADJ)

3540 noms sont rentrent en collision avec des nominatifs d'adjectifs ou de participes, soit 23.87 % des noms communs
[('abactus', 'abigo'), ('abditum', 'abditus'), ('abortus', 'aborior'), ('abruptum', 'abrumpo'), ('abscessus', 'abscedo'), ('absentia', 'absum'), ('abstantia', 'absto'), ('abstinentia', 'abstineo'), ('abundans', 'abundo'), ('abundantia', 'abundo'), ('accensi', 'accendo'), ('accensus', 'accendo'), ('accensus', 'accendo'), ('acceptum', 'acceptus'), ('accessus', 'accedo'), ('accidens', 'accido'), ('accitus', 'accio'), ('accubitus', 'accubo'), ('accursus', 'accurro'), ('acer', 'acer'), ('acerbum', 'acerbus'), ('acosmos', 'acosmos'), ('acta', 'ago'), ('actuaria', 'actuarius'), ('actuarius', 'actuarius'), ('actum', 'ago'), ('actus', 'ago'), ('acutum', 'acuo'), ('adactus', 'adigo'), ('adauctus', 'adaugeo'), ('adhaesus', 'adhaereo'), ('adiectus', 'adicio'), ('adipatum', 'adipatus'), ('adiunctum', 'adjunctus'), ('admissarius', 'admissarius'), ('admissum', 'admitto'), ('admonitus',

## Analyser (jaccard ?) les distances nominatifs / 1ere personne ?

Formes les plus communes ?

In [17]:
# Get all data

from collections import defaultdict
MOST_COMMON = defaultdict(list)

VARIATUR = {
    
}
with open("../../../forcellini-lemmas/dictionary.tsv") as f:
    header = []
    for line_no, line in enumerate(f):
        line = line.strip().split("\t")
        if line_no == 0:
            header = line
            continue
        line = dict(zip(header, line))
        lemma = line["lemma"]
        if "Description" in line:
            desc = line["Description"]
            if "," in desc:
                MOST_COMMON[lemma].extend(desc.split(", "))
            elif "/" in desc:
                MOST_COMMON[lemma].extend(desc.split("/"))

In [25]:
for lemma, variatur in MOST_COMMON.items():
    for varia in variatur:
        if varia:
            if lemma[:3] != varia[:3]:
                print(lemma, varia)
            continue
            if len(lemma) <= 3:
                if lemma[:3] != varia[:3]:
                    print("<=4", lemma, varia, lemma[:-1], varia[:len(lemma)-1])
            elif lemma[:4] != varia[:4]:
                print(lemma, varia)

ab a
abeo1 abire
abeo1 abi
Abgarus Acbarus
Abgarus Acbari
abigo abacta
abrotonum habrotonum
abrotonum habrotono
abscedo1 apscede
absinthium apsinthia
absum1 abest
absum1 abesse
absurde apsurde
acclamatio adclamationibus
acclamo adclamauit
acclinis adclinis
accognosco adcognosco
accognosco adcognoscit
accredo adcredere
accredo adcredat
accresco adcreuit
accurro adcurrit
accursus adcursu
acer2 acri
acquiesco adquiescit
acquiesco adquiescere
acquiro adquirere
acquiro adquiritur
adeo1 adire
adeo1 adiit
adeps adipes
adeps adipis
adigo adegit
adimo ademit
adipiscor adeptus
adipiscor adepti
adolescens1 adulescens
adolescens1 adulescentem
adolescens2 adulescens
adolescens2 adulescentem
adolescentia adulescentia
adolescentia adulescentiae
adolescentulus adulescentulus
adolescentulus adulescentulo
adolesco1 adulta
Aduatici Atuatucos
Aduatici Atuatucis
aeneum aheneum
aeneum ahenea
aes aere
aethra Aethra
aethra Aethrae
Afer Afro
affabilis adfabilis
affabilis adfabile
affabilitas adfabilitas
affati

Exemples marquants:

- olla aula
- olus holus
- paedico2 pedicabo
- penna pinnis
- plaustrarius2 plostrarios
- Pirithous Perithoi
- rodeus rosea
- scena scaenam
- scenicus1 scaenicorum
- scirpea sirpiam
- supparasitor subparasitabor
- supparum sipara
- Taygetus Taugeta
- uecors uaecors
- uersus3 uorsum
- Abgarus Acbarus
- abrotonum habrotonum
- absinthium apsinthia
- affectus2 adfecti
- aggestus adgestu
- anno1 adnare